# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [12]:
import numpy as np
import os
from cv2 import imread,resize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [13]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [14]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size =  30

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [15]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = range(0,30,2)#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = t.shape[0]//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,15,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #h, w, _ = image.shape
                    #image = image[int(h/2)-50:int(h/2)+50, int(w/2)-50:int(w/2)+50, : ]
                    image = resize(image,(100,100))
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - image[:,:,0].min())/(image[:,:,0].max() - image[:,:,0].min())#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - image[:,:,1].min())/(image[:,:,1].max() - image[:,:,1].min())#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - image[:,:,2].min())/(image[:,:,2].max() - image[:,:,2].min())#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if(folder_list.shape[0]%batch_size == 0):
            remaining_batch = folder_list.shape[0] - (num_batches * batch_size)
            batch_data = np.zeros((remaining_batch,15,100,100,3))
            batch_labels = np.zeros((remaining_batch,5))
            for folder in range(1,remaining_batch+1):
                img = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #h, w, _ = image.shape
                    #image = image[int(h/2)-50:int(h/2)+50, int(w/2)-50:int(w/2)+50, : ]
                    image = resize(image,(100,100))
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - image[:,:,0].min())/(image[:,:,0].max() - image[:,:,0].min())#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - image[:,:,1].min())/(image[:,:,1].max() - image[:,:,1].min())#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - image[:,:,2].min())/(image[:,:,2].max() - image[:,:,2].min())#normalise and feed in the image
                    
                    batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
            
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [16]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [17]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation ,Input ,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers,regularizers

input_layer = Input((15, 100, 100, 3))

conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 3), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 3), activation='relu')(conv_layer1)

pooling_layer1 = MaxPooling3D(pool_size=(2, 2, 2))(conv_layer2)

conv_layer3 = Conv3D(filters=24, kernel_size=(3, 3, 3), activation='relu')(pooling_layer1)

pooling_layer2 = MaxPooling3D(pool_size=(2, 2, 2))(conv_layer3)
pooling_layer3 = BatchNormalization()(pooling_layer2)
flatten_layer = Flatten()(pooling_layer3)

dense_layer1 = Dense(units=128, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.6)(dense_layer1)

output_layer = Dense(units=5, activation='softmax')(dense_layer1)

## define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer)

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [18]:
optimiser = optimizers.Adadelta(lr=0.15)#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 15, 100, 100, 3)   0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 13, 98, 98, 8)     656       
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 11, 96, 96, 16)    3472      
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 5, 48, 48, 16)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 3, 46, 46, 24)     10392     
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 1, 23, 23, 24)     0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 1, 23, 23, 24)     96        
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [19]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [20]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [21]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [22]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  ./Project_data/train ; batch size = 30
Epoch 1/30
 ./Project_data/val ; batch size = 30
23/23 [==============================] - 24s 1s/step - loss: 1.6154 - categorical_accuracy: 0.3623 - val_loss: 1.1403 - val_categorical_accuracy: 0.5833

Epoch 00001: saving model to model_init_2019-09-1613_02_12.028052/model-00001-1.61537-0.36232-1.14031-0.58333.h5
Epoch 2/30
23/23 [==============================] - 22s 962ms/step - loss: 1.0599 - categorical_accuracy: 0.5841 - val_loss: 0.9819 - val_categorical_accuracy: 0.6167

Epoch 00002: saving model to model_init_2019-09-1613_02_12.028052/model-00002-1.05993-0.58406-0.98188-0.61667.h5
Epoch 3/30
23/23 [==============================] - 22s 936ms/step - loss: 0.7416 - categorical_accuracy: 0.7101 - val_loss: 1.1627 - val_categorical_accuracy: 0.5417

Epoch 00003: saving model to model_init_2019-09-1613_02_12.028052/model-00003-0.74157-0.71014-1.16270-0.54167.h5
Epoch 4/30
23/23 [==============================] - 22

23/23 [==============================] - 22s 957ms/step - loss: 0.2224 - categorical_accuracy: 0.9391 - val_loss: 0.9139 - val_categorical_accuracy: 0.7083

Epoch 00030: saving model to model_init_2019-09-1613_02_12.028052/model-00030-0.22240-0.93913-0.91388-0.70833.h5
